# Import resources into eLab database


This script will read a csv file that contains a list of antibodies, and add them to the resources database with a fine control on which columns are processed and how.

## Tech detail

In [1]:
# the python lib for elab
import elabapi_python
# we will use the generic csv module
import csv
# we also need the json module for the metadata (extra fields) part
import json

from pathlib import Path

In [9]:
#########################
#         CONFIG        #
#########################
# replace with your instance address
API_HOST_URL = 'https://eln.ibecbarcelona.eu/api/v2/items' #IBEC
#API_HOST_URL = 'https://demo.elabftw.net/api/v2/items' #DEMO

#Available endpoints (not empty) are: 
#     apikeys, 
#     config, 
#     experiments, 
#     info, 
#     items, #this is resources database
#     experiments_templates, 
#     items_types, 
#     event, 
#     events, 
#     extra_fields_keys,
#     team_tags, 
#     teams, 
#     todolist, 
#     unfinished_steps, 
#     users

# replace with your api key
API_KEY = '1-9a2e2b47725340024b57f104dc0c520738cc81ae59d125e0674020f1a9e5b70fd7e9340de8452d262df41' #IBEC
#API_KEY = '3-945e95c845ec59ba0d69b8f6617e391a1d03d54bf9ef17fe8671756f2b6cb891914ee72a9e44eb9ce1c03' #DEMO

# this is the resource category where the entries will be created. Visit https://eln.ibecbarcelona.eu/api/v2/items_types to GET a list.

# in this example, category with id 3 corresponds to "Project CRYPTO-COOL" at IBEC
# in this example, category with id 6 corresponds to "Antibodies" at DEMO
RESOURCE_CATEGORY_ID = 45 #IBEC
# RESOURCE_CATEGORY_ID = 6 #DEMO

# parent_dir is a parent directory to our current directory `cwd`
parent_dir = Path.cwd().parent

# relative_path is a relative path to the directory where our data is stored
relative_path = 'Data/'

# filename_example is the name of the file we want to read
filename_example = 'elab_resources_import_test_antibodies_2.csv'

CSV_PATH = (parent_dir / relative_path / filename_example).resolve()

#########################
#      END CONFIG       #
#########################

In [10]:
# Configure the api client
configuration = elabapi_python.Configuration()
configuration.api_key['api_key'] = API_KEY
configuration.api_key_prefix['api_key'] = 'Authorization'
configuration.host = API_HOST_URL
configuration.debug = False
# set to True if you have a proper certificate, here it is set to False to ease the test in dev
configuration.verify_ssl = False

# create an instance of the API class
api_client = elabapi_python.ApiClient(configuration)
# fix issue with Authorization header not being properly set by the generated lib
api_client.set_default_header(header_name='Authorization', header_value=API_KEY)
    
# Load items api
itemsApi = elabapi_python.ItemsApi(api_client)

## Real stuff starts here

In [11]:
# function to build the metadata json for a row
def getMetadataFromRow(row):
    # our metadata object for one row, currently a dictionary with a key "extra_fields" holding an empty dictionary
    metadata = { 'extra_fields': {} }

    # now go over the columns (except the title/Name) and add it to our extra_fields object
    for keyval in row.items():
        field_type = 'text'

        # we don't import these columns as metadata
        # Name is the tile, Comment is in the body, and ID is the custom_id.
        if keyval[0] == 'Name' or keyval[0] == 'Comment' or keyval[0] == 'ID':
            continue
        
        # special case for url/URL column, we make it a type: url
        if keyval[0].lower() == 'url':
            field_type = 'url'
        
        if keyval[0].lower() == 'price':
            field_type = 'number'
        
        # special case for Concentration column, we use the units
        if keyval[0].lower() == 'concentration' and keyval[1]:
            split_conc = keyval[1].split()
            metadata['extra_fields'].update({keyval[0]: {'value': split_conc[0], 'type': 'number', 'unit': split_conc[1], 'units':['mg/mL', 'μg/mL']}})
        elif keyval[0].lower() == 'primary vs secondary':
            metadata['extra_fields'].update({keyval[0]: {'value': 'Primary', 'type': 'select', 'options': ['Primary', 'Secondary']}})

        elif keyval[0].lower() == 'raised in':
            metadata['extra_fields'].update({keyval[0]: {'value': keyval[1], 'type': 'select', 'options': ['Rabbit', 'Mouse']}})
        elif keyval[0].lower() == 'recognizes':
            metadata['extra_fields'].update({keyval[0]: {
                'value': keyval[1].split(', '), 'type': 'select', 'allow_multi_values': True, 'options': ['Ape', 'Chicken', 'Dog', 'Goat', 'Guinea Pig', 'Hamster', 'Human', 'Mink', 'Monkey', 'Mouse', 'Rabbit', 'Rat', 'Sheep', 'Zebrafish']}})
        else:
            metadata['extra_fields'].update({keyval[0]: {'value': keyval[1], 'type': field_type}})
    
    return json.dumps(metadata)

In [12]:
# The column "Comment" will get added to the body of the resource
def getBodyFromRow(row) -> str:
    for keyval in row.items():
        if keyval[0] == 'Comment':
            return f'<p>{keyval[1]}</p>'
    return ''


### Import is done here

In [13]:
test_file = csv.DictReader(open(CSV_PATH, newline=''), delimiter=',', quotechar='"')
print(test_file.fieldnames)
print(test_file.line_num)

['Name', 'ID', 'Vendor', 'Vendor Reference', 'URL', 'Concentration', 'Price', 'Raised in', 'Recognizes', 'Comment']
1


In [14]:
for row in test_file:
    test = body={'title': row['Name'], 'body': getBodyFromRow(row), 'custom_id': row['ID'], 'metadata': getMetadataFromRow(row)}
    print(test)

{'title': 'DDX3 (D19B4) Rabbit mAb', 'body': '<p>Use at 1:000 for WB</p>', 'custom_id': '3', 'metadata': '{"extra_fields": {"Vendor": {"value": "Cell Signaling", "type": "text"}, "Vendor Reference": {"value": "8192", "type": "text"}, "URL": {"value": "https://www.cellsignal.com/products/primary-antibodies/ddx3-d19b4-rabbit-mab/8192", "type": "url"}, "Concentration": {"value": "100", "type": "number", "unit": "\\u03bcg/mL", "units": ["mg/mL", "\\u03bcg/mL"]}, "Price": {"value": "110", "type": "number"}, "Raised in": {"value": "Rabbit", "type": "select", "options": ["Rabbit", "Mouse"]}, "Recognizes": {"value": ["Human", "Mouse", "Rat", "Monkey"], "type": "select", "allow_multi_values": true, "options": ["Ape", "Chicken", "Dog", "Goat", "Guinea Pig", "Hamster", "Human", "Mink", "Monkey", "Mouse", "Rabbit", "Rat", "Sheep", "Zebrafish"]}}}'}
{'title': 'Recombinant Anti-Calcineurin A antibody', 'body': '<p>This antibody, a unique guardian in the arsenal of immunity, stands alone in its preci

In [16]:
# Note: use encoding='utf-8-sig' in the open() call if your file has BOM (Byte Order Mark)
# Also make sure that the CSV file was saved as UTF-8 to avoid issues with special characters

with open(CSV_PATH, newline='') as csvfile: 
    
    # let's read the CSV using the standard "csv" library from python. No need for anything fancier.
    csvreader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    
    # now we loop over each row in our CSV
    for row in csvreader:

        # here we add the tag "-20°C freezer" to every row
        # the API allows setting tags during creation (POST) of a resource or experiment, so we use it here
        response = itemsApi.post_item_with_http_info(body={'category_id': RESOURCE_CATEGORY_ID, 'tags': ['-20°C freezer']})
        locationHeaderInResponse = response[2].get('Location')
        
        # that's our ID of the newly created resource
        itemId = int(locationHeaderInResponse.split('/').pop())

        # Patch the item to change its content:
        # the "Name" column becomes our title
        # the "Body" is generated from the "Comment" column content with the "getBodyFromRow()" function
        # for the "ID" column we match it to the "custom_id" property in elab
        # and the extra fields (metadata) is built with a function
        # the single line below will make all those changes at once
        itemsApi.patch_item(itemId, body={'title': row['Name'], 'body': getBodyFromRow(row), 'custom_id': row['ID'], 'metadata': getMetadataFromRow(row)})

/Users/vschastlivaia/miniforge3/envs/data-analysis/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eln.ibecbarcelona.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/vschastlivaia/miniforge3/envs/data-analysis/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eln.ibecbarcelona.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'transfer-encoding': 'chunked', 'cache-control': 'max-age=0, private, must-revalidate, no-cache, private', 'set-cookie': 'PHPSESSID=hlpo9ujgdbj0q7h3iek9tqcufumuh3jbg2h8bvonia; path=/; secure; HttpOnly; SameSite=Lax', 'date': 'Fri, 24 May 2024 15:12:05 GMT', 'strict-transport-security': 'max-age=63072000', 'x-xss-protection': '0', 'x-content-type-options': 'nosniff', 'content-security-policy': "default-src 'self' data:; script-src 'self' ; connect-src 'self' blob: https://get.elabftw.net; img-src 'self' data: blob:; style-src 'self' 'unsafe-inline'; font-src 'self' data:; object-src 'self'; base-uri 'none'; frame-ancestors 'none'", 'referrer-policy': 'no-referrer', 'permissions-policy': "autoplay 'none'; camera 'self'; document-domain 'none'; encrypted-media 'none'; fullscreen 'self'; geolocation 'none'; microphone 'self'; midi 'none'; payment 'none'; vr 'none'", 'vary': 'A

In [20]:

# Create an item with the category_id 1 (items_types ID = 1)

targetCategory = 15
response = itemsApi.post_item_with_http_info(body={'category_id': targetCategory, 'tags': ['some tag', 'another tag']})
locationHeaderInResponse = response[2].get('Location')
print(f'The newly created item is here: {locationHeaderInResponse}')
itemId = int(locationHeaderInResponse.split('/').pop())


/Users/vschastlivaia/miniforge3/envs/data-analysis/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eln.ibecbarcelona.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


The newly created item is here: https://eln.ibecbarcelona.eu/api/v2/items/41


In [21]:
itemId

41

In [23]:
# now change the title, and body and rating
itemsApi.patch_item(itemId, body={'title': 'The new title', 'body': 'Main content text', 'rating': 5})

/Users/vschastlivaia/miniforge3/envs/data-analysis/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eln.ibecbarcelona.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'transfer-encoding': 'chunked', 'cache-control': 'max-age=0, private, must-revalidate, no-cache, private', 'set-cookie': 'PHPSESSID=kho3712fh8ec65oa7ah32dfoj99h3anqe3pn5b2bt1; path=/; secure; HttpOnly; SameSite=Lax', 'date': 'Fri, 24 May 2024 15:41:41 GMT', 'strict-transport-security': 'max-age=63072000', 'x-xss-protection': '0', 'x-content-type-options': 'nosniff', 'content-security-policy': "default-src 'self' data:; script-src 'self' ; connect-src 'self' blob: https://get.elabftw.net; img-src 'self' data: blob:; style-src 'self' 'unsafe-inline'; font-src 'self' data:; object-src 'self'; base-uri 'none'; frame-ancestors 'none'", 'referrer-policy': 'no-referrer', 'permissions-policy': "autoplay 'none'; camera 'self'; document-domain 'none'; encrypted-media 'none'; fullscreen 'self'; geolocation 'none'; microphone 'self'; midi 'none'; payment 'none'; vr 'none'", 'vary': 'A

In [22]:
itemsApi.patch_item(41, body={'title': 'The new title', 'body': 'Main content text', 'rating': 5})

/Users/vschastlivaia/miniforge3/envs/data-analysis/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'eln.ibecbarcelona.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'transfer-encoding': 'chunked', 'cache-control': 'max-age=0, private, must-revalidate, no-cache, private', 'set-cookie': 'PHPSESSID=s89nnvr3k3uhkdf2q2ljk3gqfae6kh88ikllct8933; path=/; secure; HttpOnly; SameSite=Lax', 'date': 'Fri, 24 May 2024 15:22:53 GMT', 'strict-transport-security': 'max-age=63072000', 'x-xss-protection': '0', 'x-content-type-options': 'nosniff', 'content-security-policy': "default-src 'self' data:; script-src 'self' ; connect-src 'self' blob: https://get.elabftw.net; img-src 'self' data: blob:; style-src 'self' 'unsafe-inline'; font-src 'self' data:; object-src 'self'; base-uri 'none'; frame-ancestors 'none'", 'referrer-policy': 'no-referrer', 'permissions-policy': "autoplay 'none'; camera 'self'; document-domain 'none'; encrypted-media 'none'; fullscreen 'self'; geolocation 'none'; microphone 'self'; midi 'none'; payment 'none'; vr 'none'", 'vary': 'A